#### Loading libraries, importing data

In [49]:
import pandas as pd
import numpy as np
from datetime import date
import holidays
import seaborn as sns
import matplotlib.pyplot as plt

In [50]:
#load in training data
train_df = pd.read_csv(r"D:\Summer Practicum\1_Data\training_airlines.csv")
train_df.columns = train_df.columns.str.replace(' ', '')
train_df = train_df[train_df['Operating_Airline'] == 'DL'] # subset for Jetblue ONLY 
train_df.shape

C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\3075690977.py:2: DtypeWarning: Columns (86,93) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(r"D:\Summer Practicum\1_Data\training_airlines.csv")


(639217, 120)

#### Trimming and manipulating the trimmed dataframe

In [51]:
#selecting specific columns from the training data
train_df_trim = train_df[["OriginAirportID", "Flights", "FlightDate", "Month", "DayofMonth", "DayOfWeek","Operating_Airline","Origin", "OriginCityName", "OriginState","Dest","DestCityName",
"DestState","DepTime","DepDelay","DepDel15","DepartureDelayGroups","TaxiOut","TaxiIn","ArrTime","ArrDelay","ArrDel15", "ArrDelayMinutes",
"ArrivalDelayGroups",'Cancelled',"Diverted","ActualElapsedTime","AirTime","Distance","DistanceGroup",
"CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"]]

In [52]:
#filling na values in delay fields
train_df_trim[["NASDelay", "SecurityDelay", "CarrierDelay", "WeatherDelay", "LateAircraftDelay"]] = train_df_trim[["NASDelay", \
     "SecurityDelay", "CarrierDelay", "WeatherDelay", "LateAircraftDelay"]].fillna(0)

#creating a total delay field
train_df_trim["TotalDelay"] = train_df_trim[["NASDelay", "SecurityDelay", "CarrierDelay", "WeatherDelay", "LateAircraftDelay"]].sum(axis=1)

#field for if flight was cancelled or diverted
train_df_trim["CancOrDiv"] = train_df_trim["Cancelled"] + train_df_trim["Diverted"]
train_df_trim["CancOrDiv2"] = np.where((train_df_trim["Cancelled"] == 1) | train_df_trim["Diverted"] == 1, 1, 0)
train_df_trim["CancOrDiv2"] == train_df_trim["CancOrDiv"]

#field for if a flight was delayed
train_df_trim["Delayed"] = np.where((train_df_trim["ArrDelayMinutes"]) > 0, 1, 0)

#creating a region column
def state_to_region(state):
    West = ["WA", "MT", "ID", "WY", "OR", "CA", "NV", "UT", "CO", "AZ", "NM", "AK", "HI"]
    South = ["TX", "OK", "AR", "LA", "MS", "AL", "TN", "KY", "FL", "GA", "SC", "NC", "VA", "MD", "DE"]
    Northeast = ["PA", "NJ", "NY", "CT", "MA", "RI", "ME", "NH", "VT"]
    Midwest = ["ND", "SD", "NE", "KS", "MO", "IA", "MN", "WI", "IL", "IN", "OH", "MI"]
    if state in West:
        return "West"
    elif state in South:
        return "South"
    elif state in Northeast:
        return "Northeast"
    elif state in Midwest:
        return "Midwest"
    
train_df_trim["Region"] = train_df_trim["OriginState"].map(state_to_region)

C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2662193718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_trim[["NASDelay", "SecurityDelay", "CarrierDelay", "WeatherDelay", "LateAircraftDelay"]] = train_df_trim[["NASDelay", \
C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2662193718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_trim["TotalDelay"] = train_df_trim[["NASDelay", "SecurityDelay", "CarrierDelay", "WeatherDelay", "LateAircraftDelay"]].sum(axis=1)
C:\Users\iz

#### Creating the airports df that has only the top 64 airports (10000 flights +)

In [53]:

#airports_df = train_df_trim.groupby("Origin")["Flights"].value_counts().reset_index(name="Yearly_Flights")
#airports_df_trim = airports_df[airports_df["Yearly_Flights"] >= 10000]
#airports_df_trim = airports_df_trim.sort_values("Yearly_Flights", ascending=False)
#airports_df_trim.head(10)


#### Creating stats by airport

Total want to do:
Percent of flights delayed
Percent of flights diverted or cancelled


Separate sections:

When there is a delay how extreme is the delay (avg delay length given there is a delay)

When there is a delay what percentage of the delay is the fault of the airport (everything other than carrier). arrival delay minutes (early set to 0) total of all non carrier over total arrdelmins

Holiday performance indicator (given it is a holiday, average arrival delay)

In [54]:
#creating column that shows the percent of the delay that is the carriers fault

when_delayed_df = train_df_trim[train_df_trim["TotalDelay"] != 0][["NASDelay", "SecurityDelay", "CarrierDelay", 
                                                                   "WeatherDelay", "LateAircraftDelay", "Origin", "TotalDelay"]]

when_delayed_df = when_delayed_df.reset_index()

when_delayed_df["PercentAirportFault"] = ((when_delayed_df["NASDelay"] + when_delayed_df["SecurityDelay"] + when_delayed_df["WeatherDelay"]) 
                                          / when_delayed_df["TotalDelay"])

#creating a new dataframe that is the average percent that a delay is an airports fault
airport_delay_fault = when_delayed_df.groupby("Origin")["PercentAirportFault"].mean()
airport_delay_fault

Origin
ABQ    0.356511
AGS    0.028736
ALB    0.275658
ANC    0.265820
ATL    0.256989
         ...   
TUS    0.263764
TVC    0.579598
TYS    0.204589
VPS    0.235099
XNA    0.399658
Name: PercentAirportFault, Length: 150, dtype: float64

In [55]:
#percent of flights delayed and flights delayed/cancelled
airport_stats_df = train_df_trim.groupby("Origin").agg(
    percent_delayed = ("Delayed", "mean"),
    percent_diverted_cancelled = ("CancOrDiv", "mean")).reset_index() 


In [56]:
#creating the holidays column in the full dataframe
train_df_trim['FlightDate'] = pd.to_datetime(train_df_trim['FlightDate'])
train_df_trim.columns = train_df_trim.columns.str.replace(' ','')

us_holidays = holidays.US(years=range(train_df_trim['FlightDate'].min().year,train_df_trim['FlightDate'].max().year +1))
holiday_dates = {date for date, name in us_holidays.items()}

train_df_trim['Is_Holiday'] = train_df_trim['FlightDate'].dt.date.isin(holiday_dates)
train_df_trim['Is_Delayed'] = (train_df_trim['ArrDelay'] > 0)

C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2868795725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_trim['FlightDate'] = pd.to_datetime(train_df_trim['FlightDate'])
C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2868795725.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_trim['Is_Holiday'] = train_df_trim['FlightDate'].dt.date.isin(holiday_dates)
C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2868795725.py:9: SettingWithCopyWarning: 
A value is trying to be s

In [57]:
holiday_df = train_df_trim[train_df_trim["Is_Holiday"] == 1]

holiday_performance_df = holiday_df.groupby("Origin")["Delayed"].mean().reset_index()
holiday_performance_df.head(10)

,Origin,Delayed
0,ABQ,0.464286
1,ALB,0.333333
2,ANC,0.250000
3,ATL,0.348721
4,ATW,0.272727
5,AUS,0.236842
6,AVL,0.181818
7,BDL,0.242991
8,BHM,0.164179
9,BIL,0.142857


In [58]:
#how extreme is the average delay
only_delayed_flights = train_df_trim[train_df_trim["Delayed"] == 1]
how_extreme_delays = only_delayed_flights.groupby("Origin")["ArrDelayMinutes"].mean().reset_index()

#### Merging the airports information into the top 64 airports dataframe

In [59]:
#airports_df_trim

#merge1 = pd.merge(airports_df_trim[["Origin", "Yearly_Flights"]], airport_stats_df, how="left", on="Origin")

merge2 = pd.merge(airport_stats_df, airport_delay_fault, on="Origin", how="left")

merge3 = pd.merge(merge2, holiday_performance_df, on="Origin", how="left")

merge4 = pd.merge(merge3, how_extreme_delays, on="Origin", how="left")

In [60]:
Final_merge = merge4[["Origin",  "percent_delayed", "percent_diverted_cancelled", "PercentAirportFault"]]
Final_merge["HolidayPercentDelayed"] = merge4["Delayed"]
Final_merge["HowExtremeDelay"] = merge4["ArrDelayMinutes"]

C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\2297405150.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_merge["HolidayPercentDelayed"] = merge4["Delayed"]


#### Scoring

In [61]:
#Scoring for all columns with for loop
columns_of_interest = ["percent_delayed", "percent_diverted_cancelled", "PercentAirportFault", "HolidayPercentDelayed", "HowExtremeDelay"]

for column in columns_of_interest:

    quantiles = Final_merge[[column]].quantile([0.25, 0.75])
    twenty_fifth = quantiles.loc[0.25][column]
    seventy_fifth = quantiles.loc[0.75][column]

    Final_merge[("ScoreFrom_" + column)] = 0

    for i in range(len(Final_merge["Origin"])):
        value = Final_merge[column][i]

        if value <= twenty_fifth:
            points = 1
        elif value > twenty_fifth and value <= seventy_fifth:
            points = 0.75
        elif value > seventy_fifth:
            points = 0.5

        Final_merge[("ScoreFrom_" + column)][i] = points

Final_merge["TotalScore"] = Final_merge[["ScoreFrom_percent_delayed", "ScoreFrom_percent_diverted_cancelled","ScoreFrom_PercentAirportFault",
                                         "ScoreFrom_HolidayPercentDelayed", "ScoreFrom_HowExtremeDelay"]].sum(axis=1)

C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\366629263.py:22: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Final_merge[("ScoreFrom_" + column)][i] = points
C:\Users\izhang\AppData\Local\Temp\3\ipykernel_18384\366629263.py:22: Settin

In [62]:
Final_merge.sort_values(by="TotalScore", ascending=True)

,Origin,percent_delayed,percent_diverted_cancelled,PercentAirportFault,HolidayPercentDelayed,HowExtremeDelay,ScoreFrom_percent_delayed,ScoreFrom_percent_diverted_cancelled,ScoreFrom_PercentAirportFault,ScoreFrom_HolidayPercentDelayed,ScoreFrom_HowExtremeDelay,TotalScore
51,FSD,0.415020,0.025692,0.536885,0.473684,57.238095,0.50,0.5,0.5,0.50,0.50,2.50
94,MSO,0.411043,0.021472,0.359666,0.600000,52.992537,0.50,0.5,0.5,0.50,0.50,2.50
72,JAC,0.359568,0.032407,0.384738,0.320000,58.351931,0.50,0.5,0.5,0.75,0.50,2.75
47,FAR,0.220690,0.034483,0.547394,0.500000,97.375000,0.75,0.5,0.5,0.50,0.50,2.75
85,MCO,0.380374,0.026460,0.377006,0.366472,43.185698,0.50,0.5,0.5,0.50,0.75,2.75
...,...,...,...,...,...,...,...,...,...,...,...,...
147,TYS,0.138164,0.009662,0.204589,0.137931,47.167832,1.00,1.0,1.0,1.00,0.75,4.75
73,JAN,0.140146,0.002920,0.207224,0.136364,50.177083,1.00,1.0,1.0,1.00,0.75,4.75
32,DAY,0.120755,0.000000,0.185274,0.120000,45.968750,1.00,1.0,1.0,1.00,0.75,4.75
65,HSV,0.164343,0.005136,0.153684,0.166667,31.892857,1.00,1.0,1.0,1.00,1.00,5.00


#### Merging in the airport latitude and longitude data

In [63]:
#reading in airport locations

airport_locations = pd.read_csv(r"D:\Summer Practicum\0_Personal_Folders\Ben Hawley\QGIS\airport_locations.csv")

merge_locations = pd.merge(Final_merge, airport_locations, how="left", left_on="Origin", right_on="IATA")

merge_locations.to_csv("AirportScores_Locations.csv", index=False)

#### Anova to see differences in arrival delays 